# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, je vous conseil le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

# Implémenter une régression linéaire 


In [1]:
#importer vos librairies 
import pandas as pd 
import numpy as np 
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#charger les données 
#price_availability.csv
#listings_final.csv
#attention aux valeurs manquantes !!
data_price_path = '../data/price_availability.csv'

pd.set_option('display.max_columns', None)

dfPrice = pd.read_csv(data_price_path, sep=';') 
dfPrice = dfPrice.groupby('listing_id').mean()
dfPrice[['local_price']]

,local_price
listing_id,
5396,102.363985
7397,110.107632
9342,396.863014
10010,136.154856
10270,105.779221
...,...
28836096,321.428571
28838519,54.483333
28840013,112.122530


In [3]:
data_listings_path = '../data/listings_final.csv'

dfListings = pd.read_csv(data_listings_path, sep=';')
dfListings = dfListings.groupby('listing_id').mean()
dfListings

,Unnamed: 0,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
listing_id,,,,,,,,,,,,,,
56093,12,48.867284,2.358431,4,2,1,1.0,False,False,True,True,False,0.88,1.0
57207,13,48.846184,2.304455,2,1,1,1.0,False,False,True,False,False,0.87,1.0
114543,19,48.849530,2.290219,2,1,1,1.0,False,False,True,True,False,0.90,0.9
149534,9,48.866360,2.361844,4,2,1,1.0,False,False,True,True,False,1.00,0.4
164255,28,48.861398,2.364299,4,2,1,1.0,False,False,True,False,False,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28684174,662,48.879223,2.292382,5,0,1,1.0,False,True,True,False,False,1.00,1.0
28709644,745,48.872202,2.298349,4,2,1,1.0,False,True,True,False,False,1.00,1.0
28751412,88,48.880923,2.314568,2,1,0,1.0,False,True,True,False,False,1.00,1.0


In [4]:
df = pd.merge(dfPrice[['local_price']], dfListings, on=['listing_id'])
df

,local_price,Unnamed: 0,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
listing_id,,,,,,,,,,,,,,,
56093,170.000000,12,48.867284,2.358431,4,2,1,1.0,False,False,True,True,False,0.88,1.0
57207,49.952756,13,48.846184,2.304455,2,1,1,1.0,False,False,True,False,False,0.87,1.0
114543,107.374026,19,48.849530,2.290219,2,1,1,1.0,False,False,True,True,False,0.90,0.9
149534,169.000000,9,48.866360,2.361844,4,2,1,1.0,False,False,True,True,False,1.00,0.4
164255,75.876209,28,48.861398,2.364299,4,2,1,1.0,False,False,True,False,False,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28684174,725.175781,662,48.879223,2.292382,5,0,1,1.0,False,True,True,False,False,1.00,1.0
28709644,475.000000,745,48.872202,2.298349,4,2,1,1.0,False,True,True,False,False,1.00,1.0
28751412,117.000000,88,48.880923,2.314568,2,1,0,1.0,False,True,True,False,False,1.00,1.0


## Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [5]:
#définir vos variables de travail X et Y
# Y = local_price
# X = person_capacity, bedrooms, bathrooms
y = np.array(df[['local_price']])
x = np.array(df[['person_capacity', 'bedrooms', 'bathrooms']])

In [6]:
#construire l'array des prix 
y

array([[ 170.        ],
       [  49.95275591],
       [ 107.37402597],
       [ 169.        ],
       [  75.8762089 ],
       [ 284.35356201],
       [  36.58961039],
       [ 300.        ],
       [ 101.37984496],
       [  67.20833333],
       [  60.74093264],
       [  38.        ],
       [  92.        ],
       [  66.21204188],
       [  79.68041237],
       [  90.09819121],
       [ 170.        ],
       [ 115.7544757 ],
       [ 283.76115486],
       [ 144.74424552],
       [ 347.67015707],
       [ 115.        ],
       [ 105.        ],
       [ 154.65384615],
       [ 380.        ],
       [  44.16569201],
       [ 233.69170984],
       [ 146.        ],
       [ 147.96623377],
       [ 105.        ],
       [ 163.47286822],
       [ 190.        ],
       [ 180.95287958],
       [ 128.8630491 ],
       [  44.47368421],
       [ 163.28571429],
       [  94.        ],
       [  94.97619048],
       [ 218.96373057],
       [ 234.89460154],
       [ 300.78640777],
       [  62.   

En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20\ ou \ 30\%$.

In [7]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#afficher la shape de vos données 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [8]:
x_train.shape

(699, 3)

In [9]:
x_test.shape

(300, 3)

In [10]:
y_train.shape

(699, 1)

In [11]:
y_test.shape

(300, 1)

## Entraînement

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [12]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement
reg = linear_model.LinearRegression().fit(x_train, y_train)
reg

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [13]:
#afficher les coefficients
#que remarquez vous ? 
print("Coefficients beta_j :\n{}\nCoefficients INTERCEPT beta0 :\n{}".format(reg.coef_,reg.intercept_))


Coefficients beta_j :
[[29.22157692 14.61898054 86.34152526]]
Coefficients INTERCEPT beta0 :
[-46.48417235]


## Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [14]:
#faire une prediction sur X
predict = reg.predict(x_test)
predict

array([[126.868673  ],
       [534.01051785],
       [112.91948729],
       [215.20319858],
       [ 98.30050675],
       [345.3852813 ],
       [112.91948729],
       [ 83.69791037],
       [112.91948729],
       [518.73812664],
       [185.98162166],
       [ 98.30050675],
       [316.16370438],
       [171.36264112],
       [156.09024991],
       [244.4247755 ],
       [185.98162166],
       [171.36264112],
       [316.16370438],
       [214.53340375],
       [112.91948729],
       [112.91948729],
       [433.70342272],
       [112.91948729],
       [112.91948729],
       [330.76630076],
       [171.36264112],
       [ 98.30050675],
       [156.74366058],
       [112.91948729],
       [ 98.30050675],
       [142.1410642 ],
       [156.74366058],
       [259.04375604],
       [ 98.30050675],
       [171.36264112],
       [112.91948729],
       [272.32314692],
       [330.76630076],
       [ 98.30050675],
       [112.91948729],
       [112.91948729],
       [214.53340375],
       [388

Cela nous retourne un array de prédiction de prix (300 au total) pour nos x_test

In [15]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2
#que remarquez vous ? 

In [16]:
mean_squared_error(y_test, predict)

21600.006742647594

In [17]:
r2_score(y_test, predict)

0.3772870887592449

On peut dire que par $R^2$ qui est proche de 0, le modèle linéaire possède un faible pouvoir explicatif